<a href="https://colab.research.google.com/github/Sakamotto/MPCA/blob/main/ANN/NLP_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup
!cat aclImdb/train/pos/4077_10.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  2080k      0  0:00:39  0:00:39 --:--:-- 1834k
I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drin

In [2]:
import os, pathlib, shutil, random
base_dir = pathlib.Path("aclImdb")
train_dir = base_dir / "train"
val_dir = base_dir / "val"
test_dir = base_dir / "test"
for category in ["pos", "neg"]:
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname, val_dir / category / fname)

In [5]:
from tensorflow import keras
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [6]:
from transformers import pipeline
from sklearn.metrics import accuracy_score
import numpy as np
import time

In [ ]:
# se não especifidar o modelo, o default é distilbert/distilbert-base-uncased-finetuned-sst-2-english
pipeline_sentiment = pipeline("sentiment-analysis")

In [7]:
pipeline_sentiment("I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy")

[{'label': 'POSITIVE', 'score': 0.9965364933013916}]

In [ ]:
ypred = []
test_labels = []
start = time.time()
for batch, (inputs, targets) in enumerate(test_ds):
  for i in range(len(inputs)):
    yp = pipeline_sentiment(inputs[i].numpy().decode('UTF-8')[:512])
    test_labels.append(int(targets[i].numpy()==1))
    ypred.append(int(yp[0]['label']=="LABEL_1"))
  print(batch, time.time()-start, accuracy_score(test_labels, ypred))

0 11.437420845031738 0.4375
1 20.27292037010193 0.484375
2 28.233882427215576 0.4375
3 36.73539876937866 0.4453125
4 45.718985080718994 0.45625
5 53.47245407104492 0.5
6 65.40925979614258 0.5089285714285714
7 74.54191565513611 0.50390625
8 83.22569012641907 0.5034722222222222
9 100.4529197216034 0.503125
10 109.21455931663513 0.5227272727272727
11 116.75405764579773 0.5130208333333334
12 125.49844336509705 0.5192307692307693
13 133.66729378700256 0.5133928571428571
14 142.08704280853271 0.5229166666666667
15 151.87572407722473 0.525390625
16 159.4680368900299 0.5257352941176471
17 172.40357184410095 0.5208333333333334
18 181.1691596508026 0.5263157894736842
19 189.47607898712158 0.5171875
20 197.5851674079895 0.5119047619047619
21 210.09012651443481 0.5184659090909091
22 218.91885042190552 0.5190217391304348
23 226.7114531993866 0.5208333333333334
24 235.74872064590454 0.52375
25 244.3295133113861 0.5228365384615384
26 252.19159078598022 0.5219907407407407
27 263.61508536338806 0.52232